In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/LowRankVortex.jl/")

  Activating environment at `/media/mat/HDD/LowRankVortex.jl/Project.toml`


In [2]:
using Revise
using LowRankVortex
using LinearAlgebra
using Statistics
using PotentialFlow
import PotentialFlow.Plates: Plate, Points, Blobs
import PotentialFlow.Motions: reset_velocity!
import PotentialFlow.Elements
import PotentialFlow.Properties: @property
using Interpolations
using Distributions
using Test

┌ Info: Precompiling LowRankVortex [fdf095b8-6f21-420b-9d49-8e692abb03d1]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broke

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **



In [8]:
@testset "Test symmetric version of pressure for point vortices" begin
    atol = 1000*eps()
    Nv = 10
    zv = randn(Nv) + im *(0.05 .+ rand(Nv))
    Sv = randn(Nv)

    points₊ = Vortex.Point.(zv, Sv)
    points₋ = Vortex.Point.(conj(zv), -Sv)

    sys = deepcopy(vcat(points₊, points₋))

    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, points₊, 0.0)

    press = pressure_AD(sensors, deepcopy(sys), 0.0)
    @test isapprox(press_sym, press, atol = atol)
end

Test Summary:                                         | Pass  Total
Test symmetric version of pressure for point vortices |    1      1


Test.DefaultTestSet("Test symmetric version of pressure for point vortices", Any[], 1, false, false)

In [9]:
@testset "Test symmetric version of pressure for point vortices with freestream" begin
    atol = 1000*eps()
    
    U = randn(ComplexF64)
    freestream = Freestream(U)
    Nv = 10
    zv = randn(Nv) + im *(0.05 .+ rand(Nv))
    Sv = randn(Nv)

    points₊ = Vortex.Point.(zv, Sv)
    points₋ = Vortex.Point.(conj(zv), -Sv)

    sys = deepcopy(vcat(points₊, points₋))

    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, points₊, freestream, 0.0)

    press_AD = pressure_AD(sensors, deepcopy(sys), freestream, 0.0)
    press = pressure(sensors, deepcopy(sys), freestream, 0.0)

#     @test isapprox(press_AD, press, atol = atol)
    @test isapprox(press_sym, press, atol = atol)

end

sourcevels = ComplexF64[-1.006214802071761 + 0.6154150079843477im, -1.7827241325548968 + 0.11505884970584845im, -0.6253170266450545 + 1.0351996976368492im, -0.9954410999339335 + 1.7123943033250555im, -1.2286952718463175 + 1.2011025959146706im, 0.039508120239131084 + 0.6145854574154449im, -2.2661621629074533 + 1.1815304454772595im, -0.2540590492434257 + 1.3682986756573075im, -1.6394707132338953 + 1.9482763198731343im, -0.37331972875167707 + 0.1162593001280331im]
sourcevels = ComplexF64[-1.0062148020717607 - 0.6154150079843476im, -1.7827241325548968 - 0.11505884970584845im, -0.6253170266450545 - 1.0351996976368492im, -0.9954410999339335 - 1.712394303325056im, -1.2286952718463173 - 1.2011025959146706im, 0.039508120239131306 - 0.6145854574154448im, -2.2661621629074538 - 1.1815304454772595im, -0.2540590492434258 - 1.3682986756573075im, -1.639470713233895 - 1.9482763198731343im, -0.37331972875167696 - 0.1162593001280331im, -1.006214802071761 - 0.6922304909130158im, -1.7827241325548968 - 1.19

LoadError: [91mSome tests did not pass: 0 passed, 1 failed, 0 errored, 0 broken.[39m

In [19]:
@testset "Test symmetric version of pressure for regularized point vortices" begin
    atol = 1000*eps()
    Nv = 10
    zv = randn(Nv) + im *rand(Nv)
    Sv = randn(Nv)
    δ = 0.5

    blobs₊ = Vortex.Blob.(zv, Sv, δ*ones(Nv))
    blobs₋ = Vortex.Blob.(conj(zv), -Sv, δ*ones(Nv))

    sys = deepcopy(vcat(blobs₊, blobs₋))


    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, blobs₊, 0.0)

    press = pressure_AD(sensors, deepcopy(sys), 0.0)
    @test isapprox(press_sym, press, atol = atol)
end

Test Summary:                                                     | Pass  Total
Test symmetric version of pressure for regularized point vortices |    1      1


Test.DefaultTestSet("Test symmetric version of pressure for regularized point vortices", Any[], 1, false, false)